In [1]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import make_scorer, matthews_corrcoef
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv('./res/df_encoded_logically_by_lukasiak.csv')
print((df['Wzrost'].value_counts()))
scaler=MinMaxScaler()
df[df.drop('Moda',axis=1).select_dtypes(include=['number']).columns]=scaler.fit_transform(df[df.drop('Moda',axis=1).select_dtypes(include=['number']).columns])
print(df['Wzrost'].value_counts())

Wzrost
183.0    4
167.0    3
173.0    3
186.0    2
169.0    2
197.5    1
174.0    1
185.0    1
170.0    1
188.0    1
165.0    1
159.0    1
172.0    1
181.0    1
176.0    1
178.0    1
175.0    1
168.0    1
195.0    1
187.0    1
Name: count, dtype: int64
Wzrost
0.623377    4
0.207792    3
0.363636    3
0.701299    2
0.259740    2
1.000000    1
0.389610    1
0.675325    1
0.285714    1
0.753247    1
0.155844    1
0.000000    1
0.337662    1
0.571429    1
0.441558    1
0.493506    1
0.415584    1
0.233766    1
0.935065    1
0.727273    1
Name: count, dtype: int64


In [3]:
print(df['Moda'].value_counts())

Moda
0    11
3     8
1     8
2     2
Name: count, dtype: int64


In [4]:
X=df.drop('Moda',axis=1)
y=df['Moda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
print(y_test.value_counts())

Moda
0    5
3    3
1    1
Name: count, dtype: int64


In [158]:

smallest=1
biggest=len(df.columns)
models={}
scores={}
for i in range(smallest,biggest):
    model = DecisionTreeClassifier(min_samples_leaf=9)
    
    n_features=i
    
    rfe = RFE(estimator=model, n_features_to_select=n_features)
    rfe.fit(X_train, y_train)
    y_pred=rfe.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    models[i]=rfe
    scores[i]=mcc
    print(mcc)
    print([df.columns[x] for x in range(len(rfe.support_)) if rfe.support_[x] and scores[i]==0 ])

-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.04662524041201569
[]
-0.0466252404120

In [41]:
for key,value in scores.items():
    if value>0.1: print(f"{key} : {value}")

In [16]:
for key,value in models.items():
    print(f"{key} : {value.ranking_}")

1 : [135 134 133 132 131 130 129 128 127 126 125 124 123 122 121 120 119 118
 117 116 115 114 113 112 111 110 109 108 107 106 105 104 103 102 101 100
  99  98  97  96  95  94  93  92  91  90  89  88  87  86  85  84  83  82
  81  80  79  78  77  76  75  73  74  72  71  70  69  68  67  66  65  64
  63  62  61  60  59  57  58  56  55  54  53  51  52  50  49  48  47  46
  45  44  43  42  41  40  38   1  37  39  36  35  34  33  32  31  30  28
  27  29  26  25   4  14  24  23  22  21  20  19  18  17  16  15   3  13
  12  11  10   2   5   6   9   8   7]
2 : [134 133 132 131 130 129 128 127 126 125 124 123 122 121 120 119 118 117
 116 115 114 113 112 111 110 109 108 107 106 105 104 103 102 101 100  99
  98  97  96  95  91  94  93  92  90  89  88  87  86  85  84  83  81  82
  80  79  78  77  76  75  74  73  71  72  70  69  68  67  66  65  64  63
  62  61  60  59  58  56  57  55  54  53  52  51  50  49  48  47  46  45
  44  43  42  41  40  39  36   1  35  38  37  34  33  32  31  30  28  26
  29 

In [1]:
from itertools import product
from tqdm.notebook import tqdm
from math import sqrt, comb

number_of_columnes=len(df.columns)
max_depth=[None]
min_samples_leaf=list(range(1,10,1))
min_samples_split=list(range(2,23,1))
max_leaf_nodes=[None]
min_impurity_decrease=[0.0]
columns=list(range(3,int(number_of_columnes/3)))
combinations=[]
scores_params={}
for combination in tqdm(product(max_depth,min_samples_leaf,min_samples_split,max_leaf_nodes,min_impurity_decrease,columns),desc="Processing combinations", unit="combination"):
    param_dict={
        'max_depth':combination[0],
        'min_samples_leaf':combination[1],
        'min_samples_split':combination[2],
        'max_leaf_nodes':combination[3],
        'min_impurity_decrease':combination[4]
    }
    model=DecisionTreeClassifier(class_weight='balanced',**param_dict)
    rfe=RFE(estimator=model, n_features_to_select=combination[5])
    rfe.fit(X_train, y_train)
    y_pred=rfe.predict(X_test)
    mcc=matthews_corrcoef(y_test, y_pred)
    new_key=tuple(list(param_dict.items()) + [('number_of_columns', combination[5])])
    scores_params.update({new_key: mcc})
    

NameError: name 'df' is not defined

In [13]:
from itertools import product
from tqdm.notebook import tqdm
from math import sqrt, comb
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import matthews_corrcoef
from concurrent.futures import ProcessPoolExecutor

def evaluate_combination(combination):
    param_dict = {
        'max_depth': combination[0],
        'min_samples_leaf': combination[1],
        'min_samples_split': combination[2],
        'max_leaf_nodes': combination[3],
        'min_impurity_decrease': combination[4]
    }
    model = DecisionTreeClassifier(class_weight='balanced', **param_dict)
    rfe = RFE(estimator=model, n_features_to_select=combination[5])
    rfe.fit(X_train, y_train)
    y_pred = rfe.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    return tuple(list(param_dict.items()) + [('number_of_columns', combination[5])]), mcc

number_of_columnes = len(df.columns)
max_depth = [None]
min_samples_leaf = [3,4,5,6]
min_samples_split = [2,3,4,5,6,7]
max_leaf_nodes = [None]
min_impurity_decrease = [float(x) for x in np.linspace(0,1,5)]
columns = list(range(1, int(number_of_columnes / 7)))

combinations = list(product(max_depth, min_samples_leaf, min_samples_split, max_leaf_nodes, min_impurity_decrease, columns))

scores_params = {}

# Używamy ProcessPoolExecutor do zrównoleglenia obliczeń
with ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(evaluate_combination, combinations), total=len(combinations), desc="Processing combinations"))

# Aktualizujemy słownik wyników
for key, value in results:
    scores_params[key] = value


Processing combinations:   0%|          | 0/2160 [00:00<?, ?it/s]

In [9]:
for key,value in scores_params.items():
    if value>0.3 : print(f"{key} : {value}")


(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.01), ('number_of_columns', 7)) : 0.36115755925730764
(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.01), ('number_of_columns', 8)) : 0.36115755925730764
(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.01), ('number_of_columns', 9)) : 0.36115755925730764
(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.01), ('number_of_columns', 10)) : 0.36115755925730764
(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.01), ('number_of_columns', 11)) : 0.36115755925730764
(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 2), ('max_leaf_nodes', None),

In [107]:
#print(sum(1 for key,value in scores_params.items() if  value>0.3))
tmp= {}
for key,value in scores_params.items():
    if value>0.24 and key[1: 
        #tmp[(key[2],key[-1])]=value
        print(f"{key} : {value}")
        #tmp.add((key[0],key[1],key[2],key[3],key[4],value))
#print(sorted(tmp,key=lambda x:float(x[-1])) )  
#for key,value in sorted_tmp.items():
 #   print(f'{key} : {value}')

(('max_depth', None), ('min_samples_leaf', 1), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.0), ('number_of_columns', 10)) : 0.2809003238667948
(('max_depth', None), ('min_samples_leaf', 1), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.0), ('number_of_columns', 13)) : 0.2809003238667948
(('max_depth', None), ('min_samples_leaf', 1), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.0), ('number_of_columns', 15)) : 0.2809003238667948
(('max_depth', None), ('min_samples_leaf', 1), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.0), ('number_of_columns', 16)) : 0.2809003238667948
(('max_depth', None), ('min_samples_leaf', 1), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.0), ('number_of_columns', 26)) : 0.2809003238667948
(('max_depth', None), ('min_samples_leaf', 1), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_

In [101]:
values_dict={}
for key,value in scores_params.items():
    
    try:
        values_dict[float(value)]+=1
    except KeyError:
        values_dict[float(value)]=1
print(values_dict)

{-0.14744195615489714: 40, -0.148969892882781: 6, -0.060192926542884606: 15, 0.019360077316559157: 51, -0.07744030926623663: 30, 0.2809003238667948: 21, 0.18766297265136728: 15, 0.15217391304347827: 8, 0.04662524041201569: 1774, 0.04256282653793743: 17, 0.1404501619333974: 18, -0.18057877962865382: 6, -0.1276884796138123: 9, 0.23643312187173018: 4, -0.06255432421712243: 5, -0.2391304347826087: 4, 0.020064308847628203: 2, -0.3263766828841098: 346, 0.045501575519329006: 7, 0.16051447078102563: 1, 0.059108280467932546: 7, -0.15488061853247326: 5, -0.24227185592617448: 3, 0.1936007731655916: 183, 0.367841469014624: 62, -0.19702760155977517: 1, 0.05572782125753528: 40, -0.3830654388414369: 840, 0.07372097807744857: 20, -0.17391304347826086: 430, 0.28625128703833574: 164, 0.36115755925730764: 266}


In [17]:
param_set=set()
param_set_2=set()
for key,value in scores_params.items():
    if value>0.36 and key[4][1]<0.2 and key[-1][1]<4: param_set.add((key[0],key[1],key[2],key[3],key[4],key[5],float(value))); param_set_2.add(key[0]); param_set_2.add(key[1]); param_set_2.add(key[2]); param_set_2.add(key[3]); param_set_2.add(key[4]);param_set_2.add(key[5]) 
#print(sorted(param_set_2,key=lambda x: (x[0],x[1])))
print(*sorted(param_set,key=lambda x:x[-1]),sep='\n\n')

(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 7), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.0), ('number_of_columns', 3), 0.36115755925730764)

(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 4), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.0), ('number_of_columns', 2), 0.36115755925730764)

(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 6), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.0), ('number_of_columns', 2), 0.36115755925730764)

(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.0), ('number_of_columns', 3), 0.36115755925730764)

(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 2), ('max_leaf_nodes', None), ('min_impurity_decrease', 0.0), ('number_of_columns', 2), 0.36115755925730764)

(('max_depth', None), ('min_samples_leaf', 6), ('min_samples_split', 6), ('max_leaf_nodes', None), ('min_

In [75]:
column_set=set()
for key,value in scores_params.items():
    if value>0.3: column_set.add(value)
print(sorted(column_set))
print(sum(1 for key,value in scores_params.items() if value>0.3))

[np.float64(0.3423265984407288)]
216


In [28]:
model2=DecisionTreeClassifier(max_depth=None,min_samples_leaf=6,min_samples_split=5,max_leaf_nodes=None)
rfe = RFE(estimator=model2, n_features_to_select=7)
rfe.fit(X_train, y_train)
for i in range(len(rfe.support_)):
    if rfe.support_[i]==True:
        print(X_train.columns[i])
print(matthews_corrcoef(rfe.predict(X_test),y_test))
print(rfe.predict(X_test))
print(y_test)

Zajecie__sluchanie muzyki
Wakacje__Podrozowanie
Wakacje__Praca
Zwierzak__maly gryzon (kawia, mysz...)
Zwierzak__pies
Plec__kobieta
Plec__mezczyzna
0.023312620206007845
[0 0 0 0 1 3 1 0 0]
14    0
7     3
5     3
2     0
26    0
20    3
12    0
13    0
18    1
Name: Moda, dtype: int64


In [76]:
print(rfe.predict(X_test))
print(y_test.value_counts())

[3 0 1 1 2 1 1 0 0]
Moda
0    3
3    2
1    2
2    2
Name: count, dtype: int64


In [46]:
model3=DecisionTreeClassifier(min_samples_leaf=3)
model3.fit(X_train, y_train)
y_pred=model3.predict(X_test)
mcc=matthews_corrcoef(y_test, y_pred)
print(model3.feature_importances_)
print(mcc)
feature_importances={}
for i in range(len(model3.feature_importances_)):
    feature_importances[X_train.columns[i]]=model3.feature_importances_[i]
for key,value in feature_importances.items():
    if value>0:print(f'\n{key} : {value}')

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.03698567 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.03236246 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.30147172 0.         0.         0.         

In [40]:
print(type(model3.feature_importances_))

<class 'numpy.ndarray'>
